In [ ]:
basedir = '/home/jun/run/'

In [ ]:
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from matplotlib.lines import Line2D


def find_top5_asymmetry(data: np.ndarray):
    asymmetry_matrix = np.abs(np.triu(data, k=1) - np.tril(data, k=-1).T)
    asymmetry_combinations = np.triu(asymmetry_matrix, k=1)
    top5_indices = np.unravel_index(np.argsort(
        asymmetry_combinations, axis=None)[-5:], asymmetry_combinations.shape)
    return top5_indices


def draw_heatmap(title: str, data: np.ndarray, eventList: list):
    xticklabels = eventList
    yticklabels = list(reversed(eventList))
    matrix_flipped = np.flipud(data)
    ax = sns.heatmap(matrix_flipped, xticklabels=xticklabels,
                     yticklabels=yticklabels)

    plt.gcf().set_size_inches(10, 8)

    line = Line2D([matrix_flipped.shape[1], 0], [
        0, matrix_flipped.shape[0]], color='red')
    plt.gca().add_line(line)
    plt.title(title)
    plt.show()


def analyze(bench):
    map = {}
    top_pairs = []

    path = os.path.join(basedir, bench)
    os.chdir(path)

    with open(bench+'.out', 'r') as f:
        lines = []
        text = ""
        for line in f:
            if line.startswith('recorded events:'):
                eventList = line.split(":")[1].strip().split(" ")
            lines.append(line)
        text = "".join(lines)

    match1 = re.search(r"Executed instructions: (\d+)", text)
    if match1:
        executed_instructions = int(match1.group(1))
        i = executed_instructions / 1000
    else:
        print("No executed_instructions found.")

    print(bench + ": ", executed_instructions)

    match3 = re.search(r"all events count: (\d+)", text)
    if match3:
        event_count = int(match3.group(1))
    else:
        print("No event_count found.")

    assert len(eventList) == event_count, "event count mismatch"
    map[bench] = event_count
    del event_count

    fd = 'bc16V1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m16v1 = np.array(data_list, np.uint64) / i

    fd = 'bc16V2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m16v2 = np.array(data_list, np.uint64) / i

    top5_asymmetry_results = {}
    top5_asymmetry_results[bench] = find_top5_asymmetry(m16v2)
    indices = top5_asymmetry_results[bench]

    top_pairs = [(eventList[i], eventList[j]) for i, j in zip(*indices)]

    fd = 'bc32V1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m32v1 = np.array(data_list, np.uint64) / i

    fd = 'bc32V2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m32v2 = np.array(data_list, np.uint64) / i

    fd = 'bc64V1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m64v1 = np.array(data_list, np.uint64) / i

    fd = 'bc64V2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m64v2 = np.array(data_list, np.uint64) / i

    fd = 'bc128V1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m128v1 = np.array(data_list, np.uint64) / i

    fd = 'bc128V2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m128v2 = np.array(data_list, np.uint64) / i

    fd = 'bc256V1.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m256v1 = np.array(data_list, np.uint64) / i

    fd = 'bc256V2.txt'
    with open(fd, 'r') as file:
        data = file.read()
    data_list = eval(data)
    m256v2 = np.array(data_list, np.uint64) / i

    m32v1 = m32v1 + m16v1
    m32v2 = m32v2 + m16v2
    m64v1 = m64v1 + m32v1
    m64v2 = m64v2 + m32v2
    m128v1 = m128v1 + m64v1
    m128v2 = m128v2 + m64v2
    m256v1 = m256v1 + m128v1
    m256v2 = m256v2 + m128v2

    max_index = np.unravel_index(np.argmax(m16v2, axis=None), m16v2.shape)
    top16 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m32v2, axis=None), m32v2.shape)
    top32 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m64v2, axis=None), m64v2.shape)
    top64 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m128v2, axis=None), m128v2.shape)
    top128 = (eventList[max_index[0]], eventList[max_index[1]])

    max_index = np.unravel_index(np.argmax(m256v2, axis=None), m256v2.shape)
    top256 = (eventList[max_index[0]], eventList[max_index[1]])

    tops = zip("top16, top32, top64, top128, top256".split(", "),
               [top16, top32, top64, top128, top256])
    m = zip("m16v2, m32v2, m64v2, m128v2, m256v2".split(
        ", "), [m16v2, m32v2, m64v2, m128v2, m256v2])

    return {key: value for (key, value) in m}, top_pairs, dict(tops), eventList

In [ ]:
# benchlist = '''polybench poorcache crc32 cubic edn huffbench matmultint md5sum minver nbody nettleaes
#     nettlesha256 nsichneu picojpeg primecount qrduino slre st statemate tarfind ud wikisort'''.split()
benchlist = '''crc32'''.split()
tops = {}
matrixs = {}
top_pairs = {}
events = {}

for bench in benchlist:
    matrixs[bench], top_pairs[bench], tops[bench], events[bench] = analyze(
        bench)

for bench in benchlist:
    draw_heatmap(bench+", 16 cycle", matrixs[bench]['m16v2'], events[bench])